In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from shapely.wkt import loads as wkt_loads
import tifffile as tiff
import os
import random
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Cropping2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from sklearn.metrics import jaccard_similarity_score
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
from collections import defaultdict

from keras import __version__
print(__version__)

Using TensorFlow backend.


2.0.5


In [2]:
K.set_image_data_format('channels_first')


In [3]:
N_Cls = 10
inDir = '/home/ubuntu/data/dstl'
DF = pd.read_csv(inDir + '/train_wkt_v4.csv')
GS = pd.read_csv(inDir + '/grid_sizes.csv', names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)
SB = pd.read_csv(os.path.join(inDir, 'sample_submission.csv'))
ISZ = 160
smooth = 1e-12

In [4]:

def _convert_coordinates_to_raster(coords, img_size, xymax):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    Xmax, Ymax = xymax
    H, W = img_size
    W1 = 1.0 * W * W / (W + 1)
    H1 = 1.0 * H * H / (H + 1)
    xf = W1 / Xmax
    yf = H1 / Ymax
    coords[:, 1] *= yf
    coords[:, 0] *= xf
    coords_int = np.round(coords).astype(np.int32)
    return coords_int


def _get_xmax_ymin(grid_sizes_panda, imageId):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    xmax, ymin = grid_sizes_panda[grid_sizes_panda.ImageId == imageId].iloc[0, 1:].astype(float)
    return (xmax, ymin)


def _get_polygon_list(wkt_list_pandas, imageId, cType):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    df_image = wkt_list_pandas[wkt_list_pandas.ImageId == imageId]
    multipoly_def = df_image[df_image.ClassType == cType].MultipolygonWKT
    polygonList = None
    if len(multipoly_def) > 0:
        assert len(multipoly_def) == 1
        polygonList = wkt_loads(multipoly_def.values[0])
    return polygonList


def _get_and_convert_contours(polygonList, raster_img_size, xymax):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    perim_list = []
    interior_list = []
    if polygonList is None:
        return None
    for k in range(len(polygonList)):
        poly = polygonList[k]
        perim = np.array(list(poly.exterior.coords))
        perim_c = _convert_coordinates_to_raster(perim, raster_img_size, xymax)
        perim_list.append(perim_c)
        for pi in poly.interiors:
            interior = np.array(list(pi.coords))
            interior_c = _convert_coordinates_to_raster(interior, raster_img_size, xymax)
            interior_list.append(interior_c)
    return perim_list, interior_list


def _plot_mask_from_contours(raster_img_size, contours, class_value=1):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    img_mask = np.zeros(raster_img_size, np.uint8)
    if contours is None:
        return img_mask
    perim_list, interior_list = contours
    cv2.fillPoly(img_mask, perim_list, class_value)
    cv2.fillPoly(img_mask, interior_list, 0)
    return img_mask


def generate_mask_for_image_and_class(raster_size, imageId, class_type, grid_sizes_panda=GS, wkt_list_pandas=DF):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    xymax = _get_xmax_ymin(grid_sizes_panda, imageId)
    polygon_list = _get_polygon_list(wkt_list_pandas, imageId, class_type)
    contours = _get_and_convert_contours(polygon_list, raster_size, xymax)
    mask = _plot_mask_from_contours(raster_size, contours, 1)
    return mask


def M(image_id):
    # __author__ = amaia
    # https://www.kaggle.com/aamaia/dstl-satellite-imagery-feature-detection/rgb-using-m-bands-example
    filename = os.path.join(inDir, 'sixteen_band', '{}_M.tif'.format(image_id))
    img = tiff.imread(filename)
    img = np.rollaxis(img, 0, 3)
    img = cv2.resize(img, (835, 835)) 
    return img

# def A(image_id):
#     filename = os.path.join(inDir, 'sixteen_band', '{}_A.tif'.format(image_id))
#     img = tiff.imread(filename)
#     img = np.rollaxis(img, 0, 3)    
#     return img

def P(image_id):
    filename = os.path.join(inDir, 'sixteen_band', '{}_P.tif'.format(image_id))
    img = tiff.imread(filename) 
    img = cv2.resize(img, (835, 835))
    return img

# def RGB(image_id):
#     filename = os.path.join(inDir, 'three_band', '{}.tif'.format(image_id))
#     img = tiff.imread(filename)
#     img = np.rollaxis(img, 0, 3)    
#     return img


    

def stretch_n(bands, lower_percent=5, higher_percent=95):
    # added if to deal with P band which has only 1 layer
    if len(bands.shape) < 3: 
        bands = np.expand_dims(bands, axis=3) 
    out = np.zeros_like(bands).astype(np.float32)
    n = bands.shape[2]
    for i in range(n):
        a = 0  # np.min(band)
        b = 1  # np.max(band)
        c = np.percentile(bands[:, :, i], lower_percent)
        d = np.percentile(bands[:, :, i], higher_percent)
        t = a + (bands[:, :, i] - c) * (b - a) / (d - c)
        t[t < a] = a
        t[t > b] = b
        out[:, :, i] = t

    return out.astype(np.float32)


def jaccard_coef(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)


def stick_all_train():
    print "let's stick all imgs together"
    s = 835

    x = np.zeros((5 * s, 5 * s, 9))
    y = np.zeros((5 * s, 5 * s, N_Cls))

    ids = sorted(DF.ImageId.unique())
    print len(ids)
    for i in range(5):
        for j in range(5):
            id = ids[5 * i + j]

            img_M = M(id)
            img_P = np.expand_dims(P(id), axis=3)
            img = np.append(img_P, img_M, axis=2)
#             img = M(id)
            img = stretch_n(img)
            print img.shape, id, np.amax(img), np.amin(img)
            x[s * i:s * i + s, s * j:s * j + s, :] = img[:s, :s, :]
            for z in range(N_Cls):
                y[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class(
                    (img.shape[0], img.shape[1]), id, z + 1)[:s, :s]

    print np.amax(y), np.amin(y)

    np.save(inDir + '/data/x_trn_%d' % N_Cls, x)
    np.save(inDir + '/data/y_trn_%d' % N_Cls, y)

    
def get_patches(img, msk, amt=10000, aug=True):
    is2 = int(1.0 * ISZ)
    xm, ym = img.shape[0] - is2, img.shape[1] - is2

    x, y = [], []

    tr = [0.4, 0.1, 0.1, 0.15, 0.3, 0.95, 0.1, 0.05, 0.001, 0.005]
    for i in range(amt):
        xc = random.randint(0, xm)
        yc = random.randint(0, ym)

        im = img[xc:xc + is2, yc:yc + is2]
        ms = msk[xc:xc + is2, yc:yc + is2]

        for j in range(N_Cls):
            sm = np.sum(ms[:, :, j])
            if 1.0 * sm / is2 ** 2 > tr[j]:
                if aug:
                    if random.uniform(0, 1) > 0.5:
                        im = im[::-1]
                        ms = ms[::-1]
                    if random.uniform(0, 1) > 0.5:
                        im = im[:, ::-1]
                        ms = ms[:, ::-1]

                x.append(im)
                y.append(ms)

    x, y = 2 * np.transpose(x, (0, 3, 1, 2)) - 1, np.transpose(y, (0, 3, 1, 2))
    print x.shape, y.shape, np.amax(x), np.amin(x), np.amax(y), np.amin(y)
    return x, y


In [5]:
stick_all_train()

let's stick all imgs together
25


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:160: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


(835, 835, 9) 6010_1_2 1.0 0.0
(835, 835, 9) 6010_4_2 1.0 0.0
(835, 835, 9) 6010_4_4 1.0 0.0
(835, 835, 9) 6040_1_0 1.0 0.0
(835, 835, 9) 6040_1_3 1.0 0.0
(835, 835, 9) 6040_2_2 1.0 0.0
(835, 835, 9) 6040_4_4 1.0 0.0
(835, 835, 9) 6060_2_3 1.0 0.0
(835, 835, 9) 6070_2_3 1.0 0.0
(835, 835, 9) 6090_2_0 1.0 0.0
(835, 835, 9) 6100_1_3 1.0 0.0
(835, 835, 9) 6100_2_2 1.0 0.0
(835, 835, 9) 6100_2_3 1.0 0.0
(835, 835, 9) 6110_1_2 1.0 0.0
(835, 835, 9) 6110_3_1 1.0 0.0
(835, 835, 9) 6110_4_0 1.0 0.0
(835, 835, 9) 6120_2_0 1.0 0.0
(835, 835, 9) 6120_2_2 1.0 0.0
(835, 835, 9) 6140_1_2 1.0 0.0
(835, 835, 9) 6140_3_1 1.0 0.0
(835, 835, 9) 6150_2_3 1.0 0.0
(835, 835, 9) 6160_2_1 1.0 0.0
(835, 835, 9) 6170_0_4 1.0 0.0
(835, 835, 9) 6170_2_4 1.0 0.0
(835, 835, 9) 6170_4_1 1.0 0.0
1.0 0.0


In [6]:
def make_val():
    print "let's pick some samples for validation"
    img = np.load(inDir + '/data/x_trn_%d.npy' % N_Cls)
    msk = np.load(inDir + '/data/y_trn_%d.npy' % N_Cls)
    x, y = get_patches(img, msk, amt=5000)

    np.save(inDir + '/data/x_tmp_%d' % N_Cls, x)
    np.save(inDir + '/data/y_tmp_%d' % N_Cls, y)

In [7]:
make_val()

let's pick some samples for validation
(1691, 9, 160, 160) (1691, 10, 160, 160) 1.0 -1.0 1.0 0.0


In [11]:

def get_unet():
    inputs = Input((9, ISZ, ISZ))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
#     crop1 = Cropping2D(cropping=((16, 16), (16, 16)))(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=1)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=1)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=1)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = Cropping2D([(5, 5), (5, 5)], data_format='channels_first')(conv9)
    
    conv10 = Conv2D(N_Cls, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model


def calc_jacc(model):
    img = np.load(inDir + '/data/x_tmp_%d.npy' % N_Cls)
    msk = np.load(inDir + '/data/y_tmp_%d.npy' % N_Cls)

    prd = model.predict(img, batch_size=4)
    print prd.shape, msk.shape
    avg, trs = [], []

    for i in range(N_Cls):
        t_msk = msk[:, i, :, :]
        t_prd = prd[:, i, :, :]
        t_msk = t_msk.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])
        t_prd = t_prd.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])

        m, b_tr = 0, 0
        for j in range(10):
            tr = j / 10.0
            pred_binary_mask = t_prd > tr

            jk = jaccard_similarity_score(t_msk, pred_binary_mask)
            if jk > m:
                m = jk
                b_tr = tr
        print i, m, b_tr
        avg.append(m)
        trs.append(b_tr)

    score = sum(avg) / 10.0
    return score, trs


def mask_for_polygons(polygons, im_size):
    # __author__ = Konstantin Lopuhin
    # https://www.kaggle.com/lopuhin/dstl-satellite-imagery-feature-detection/full-pipeline-demo-poly-pixels-ml-poly
    img_mask = np.zeros(im_size, np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask


def mask_to_polygons(mask, epsilon=5, min_area=1.):
    # __author__ = Konstantin Lopuhin
    # https://www.kaggle.com/lopuhin/dstl-satellite-imagery-feature-detection/full-pipeline-demo-poly-pixels-ml-poly

    # first, find contours with cv2: it's much faster than shapely
    image, contours, hierarchy = cv2.findContours(
        ((mask == 1) * 255).astype(np.uint8),
        cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_KCOS)
    # create approximate contours to have reasonable submission size
    approx_contours = [cv2.approxPolyDP(cnt, epsilon, True)
                       for cnt in contours]
    if not contours:
        return MultiPolygon()
    # now messy stuff to associate parent and child contours
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    # http://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(approx_contours[idx])
    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(approx_contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    # approximating polygons might have created invalid ones, fix them
    all_polygons = MultiPolygon(all_polygons)
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
        # need to keep it a Multi throughout
        if all_polygons.type == 'Polygon':
            all_polygons = MultiPolygon([all_polygons])
    return all_polygons


def get_scalers(im_size, x_max, y_min):
    # __author__ = Konstantin Lopuhin
    # https://www.kaggle.com/lopuhin/dstl-satellite-imagery-feature-detection/full-pipeline-demo-poly-pixels-ml-poly
    h, w = im_size  # they are flipped so that mask_for_polygons works correctly
    h, w = float(h), float(w)
    w_ = 1.0 * w * (w / (w + 1))
    h_ = 1.0 * h * (h / (h + 1))
    return w_ / x_max, h_ / y_min


def train_net():
    print "start train net"
    x_val, y_val = np.load(inDir + '/data/x_tmp_%d.npy' % N_Cls), np.load(inDir + '/data/y_tmp_%d.npy' % N_Cls)
    img = np.load(inDir + '/data/x_trn_%d.npy' % N_Cls)
    msk = np.load(inDir + '/data/y_trn_%d.npy' % N_Cls)

    x_trn, y_trn = get_patches(img, msk)

    model = get_unet()
    model.load_weights(inDir + '/weights/unet_PandM_jk0.8318')
    model_checkpoint = ModelCheckpoint(inDir + '/weights/unet_tmp.hdf5', monitor='loss', save_best_only=True)
    for i in range(1):
        model.fit(x_trn, y_trn, batch_size=64, epochs=1, verbose=1, shuffle=True,
                  callbacks=[model_checkpoint], validation_data=(x_val, y_val))
        del x_trn
        del y_trn
        x_trn, y_trn = get_patches(img, msk)
        score, trs = calc_jacc(model)
        print 'val jk', score
        model.save_weights(inDir + '/weights/unet_PandM_1600_jk%.4f' % score)

    return model


def predict_id(id, model, trs):
    img_M = M(id)
    img_P = np.expand_dims(P(id), axis=3)
    img = np.append(img_P, img_M, axis=2)
#     img = M(id)
    x = stretch_n(img)

    cnv = np.zeros((960, 960, 9)).astype(np.float32)
    prd = np.zeros((N_Cls, 960, 960)).astype(np.float32)
    cnv[:img.shape[0], :img.shape[1], :] = x

    for i in range(0, 6):
        line = []
        for j in range(0, 6):
            line.append(cnv[i * ISZ:(i + 1) * ISZ, j * ISZ:(j + 1) * ISZ])

        x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
        tmp = model.predict(x, batch_size=4)
        for j in range(tmp.shape[0]):
            prd[:, i * ISZ:(i + 1) * ISZ, j * ISZ:(j + 1) * ISZ] = tmp[j]

    # trs = [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1]
    for i in range(N_Cls):
        prd[i] = prd[i] > trs[i]

    return prd[:, :img.shape[0], :img.shape[1]]


def predict_test(model, trs):
    print "predict test"
    for i, id in enumerate(sorted(set(SB['ImageId'].tolist()))):
        msk = predict_id(id, model, trs)
        np.save(inDir + '/msk/10_%s' % id, msk)
        if i % 100 == 0: print i, id


def make_submit():
    print "make submission file"
    df = pd.read_csv(os.path.join(inDir, 'sample_submission.csv'))
    print df.head()
    for idx, row in df.iterrows():
        id = row[0]
        kls = row[1] - 1

        msk = np.load(inDir + '/msk/10_%s.npy' % id)[kls]
        pred_polygons = mask_to_polygons(msk)
        x_max = GS.loc[GS['ImageId'] == id, 'Xmax'].as_matrix()[0]
        y_min = GS.loc[GS['ImageId'] == id, 'Ymin'].as_matrix()[0]

        x_scaler, y_scaler = get_scalers(msk.shape, x_max, y_min)

        scaled_pred_polygons = shapely.affinity.scale(pred_polygons, xfact=1.0 / x_scaler, yfact=1.0 / y_scaler,
                                                      origin=(0, 0, 0))

        df.iloc[idx, 2] = shapely.wkt.dumps(scaled_pred_polygons)
        if idx % 100 == 0: print idx
    print df.head()
    df.to_csv(inDir + '/subm/subm_unet_PandM_croplayer.csv', index=False)
    return df


def check_predict(id='6120_2_3'):
    model = get_unet()
#     model.load_weights(inDir + '/weights/unet_10_jk0.7878')

    msk = predict_id(id, model, [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1])
    img = M(id)

    plt.figure()
    ax1 = plt.subplot(131)
    ax1.set_title('image ID:6120_2_3')
    ax1.imshow(img[:, :, 5], cmap=plt.get_cmap('gist_ncar'))
    ax2 = plt.subplot(132)
    ax2.set_title('predict bldg pixels')
    ax2.imshow(msk[0], cmap=plt.get_cmap('gray'))
    ax3 = plt.subplot(133)
    ax3.set_title('predict bldg polygones')
    ax3.imshow(mask_for_polygons(mask_to_polygons(msk[0], epsilon=1), img.shape[:2]), cmap=plt.get_cmap('gray'))

    plt.show()



In [12]:
ids = sorted(DF.ImageId.unique())
P(ids[0]).shape

# making the images bigger crashed kernel

(835, 835)

In [13]:
model = train_net()

start train net
(3421, 9, 160, 160) (3421, 10, 160, 160) 1.0 -1.0 1.0 0.0


ValueError: Error when checking target: expected conv2d_19 to have shape (None, 10, 150, 150) but got array with shape (3421, 10, 160, 160)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 9, 160, 160)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 160, 160)      2624      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 160, 160)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 80, 80)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 80, 80)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 80, 80)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 40, 40)        0         
__________

In [16]:
model = train_net()

start train net
(3572, 9, 160, 160) (3572, 10, 160, 160) 1.0 -1.0 1.0 0.0
Train on 3572 samples, validate on 1054 samples
Epoch 1/2
3572/3572 [==============================] - 350s - loss: 0.0823 - jaccard_coef: 0.2405 - jaccard_coef_int: 0.2330 - acc: 0.9708 - val_loss: 0.0826 - val_jaccard_coef: 0.2369 - val_jaccard_coef_int: 0.2284 - val_acc: 0.9711
Epoch 2/2
3572/3572 [==============================] - 346s - loss: 0.0702 - jaccard_coef: 0.2815 - jaccard_coef_int: 0.2777 - acc: 0.9750 - val_loss: 0.0666 - val_jaccard_coef: 0.2997 - val_jaccard_coef_int: 0.3093 - val_acc: 0.9759
(3597, 9, 160, 160) (3597, 10, 160, 160) 1.0 -1.0 1.0 0.0
(1054, 10, 160, 160) (1054, 10, 160, 160)
0 0.84526039736 0.4
1 0.682942360451 0.2
2 0.873292408425 0.5
3 0.418100029557 0.2
4 0.538577771111 0.4
5 0.910134318459 0.6
6 0.974764709833 0.5
7 0.974012077782 0.5
8 0.987090844402 0.2
9 0.937233159393 0.3
val jk 0.814140807677


In [12]:
# model = train_net()
score, trs = calc_jacc(model)

(1152, 10, 160, 160) (1152, 10, 160, 160)
0 0.86674958484 0.4
1 0.732231987847 0.4
2 0.868463690503 0.8
3 0.560665796097 0.5
4 0.519840148961 0.4
5 0.857421696638 0.9
6 0.973925637859 0.9
7 0.978617722978 0.8
8 0.990923394097 0.3
9 0.969455295139 0.3


In [19]:
predict_test(model, trs)

predict test
0 6010_0_0
100 6050_1_0
200 6090_1_1
300 6130_2_4
400 6170_4_0


In [22]:
df = make_submit()

Self-intersection at or near point 679.29999999999995 751.29999999999995
Self-intersection at or near point 163.24000000000001 308


make submission file
    ImageId  ClassType                                    MultipolygonWKT
0  6120_2_4          1  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
1  6120_2_4          2  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
2  6120_2_4          3  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
3  6120_2_4          4  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
4  6120_2_4          5  POLYGON ((0 0, 0.009188 0, 0.009188 -0.0090399...
0


Self-intersection at or near point 345.36619718309856 632.05633802816897
Self-intersection at or near point 834 0
Self-intersection at or near point 812.14754098360652 525.47213114754095
Self-intersection at or near point 194.18181818181819 570.63636363636363
Self-intersection at or near point 162.54794520547946 162.87671232876713
Self-intersection at or near point 799.14285714285711 469.28571428571428
Self-intersection at or near point 184.14076246334312 65.52785923753666
Self-intersection at or near point 830.44604316546759 411.136690647482
Self-intersection at or near point 833.375 689.5
Self-intersection at or near point 829.61403508771934 700.89122807017543
Self-intersection at or near point 824.60000000000002 192.80000000000001
Self-intersection at or near point 193.55555555555554 346.84848484848487
Self-intersection at or near point 789.6875 455
Self-intersection at or near point 513.96284829721367 27.839009287925698
Self-intersection at or near point 819.84246575342468 769.2671

100


Self-intersection at or near point 340.58018867924528 14.174528301886792
Self-intersection at or near point 831.90322580645159 691.51612903225805
Self-intersection at or near point 475.67676767676767 813.93939393939399
Self-intersection at or near point 831.32989690721649 519.13402061855675
Self-intersection at or near point 588.23913043478262 487.60869565217394
Self-intersection at or near point 721.17391304347825 541.21739130434787
Self-intersection at or near point 790.19047619047615 337.71428571428572
Self-intersection at or near point 486.5625 535.125
Self-intersection at or near point 548.56666666666672 565.45000000000005
Self-intersection at or near point 737.27692307692303 72.784615384615378
Self-intersection at or near point 700.63687150837984 561.49162011173189
Self-intersection at or near point 274.4153846153846 315.6307692307692
Self-intersection at or near point 284.45161290322579 436.74193548387098
Self-intersection at or near point 2 197
Self-intersection at or near poin

...

4100


Self-intersection at or near point 832.27027027027032 586.21621621621625
Self-intersection at or near point 664.23529411764707 291.94117647058823
Self-intersection at or near point 732 84.760000000000005
Self-intersection at or near point 317.24324324324323 23.216216216216218
Self-intersection at or near point 832.15352697095432 433.70539419087135
Self-intersection at or near point 635.9677419354839 692.64516129032256
Self-intersection at or near point 774.07692307692309 31.615384615384613
Self-intersection at or near point 673 479
Self-intersection at or near point 330.78947368421052 414.63157894736844
Self-intersection at or near point 466.7037037037037 361.18518518518516
Self-intersection at or near point 820.09049773755657 644.12669683257923
Self-intersection at or near point 792.18848167539272 6.8167539267015709
Self-intersection at or near point 816.82258064516134 611.77419354838707
Self-intersection at or near point 787.86842105263156 377.98684210526318
Self-intersection at or n

4200


Self-intersection at or near point 817.9885057471264 380.05747126436779
Self-intersection at or near point 813.82051282051282 432.76923076923077
Self-intersection at or near point 676 168
Self-intersection at or near point 832.20699708454811 645.11661807580174
Self-intersection at or near point 473.97058823529414 11.029411764705882
Self-intersection at or near point 720.62436548223354 321.11675126903555
Self-intersection at or near point 550.00868306801738 743.76266280752532
Self-intersection at or near point 616.70443349753691 184.97536945812809
Self-intersection at or near point 641.82352941176475 437.1764705882353
Self-intersection at or near point 733.98882681564248 266.39664804469271
Self-intersection at or near point 711.13043478260875 695.39130434782612
Self-intersection at or near point 383.4736842105263 777.26315789473688
Self-intersection at or near point 831.71573604060916 135.49238578680203
Self-intersection at or near point 366.12162162162161 184.98108108108107
Self-inters

    ImageId  ClassType                                    MultipolygonWKT
0  6120_2_4          1  MULTIPOLYGON (((0.0062961060776650 -0.00714310...
1  6120_2_4          2                           GEOMETRYCOLLECTION EMPTY
2  6120_2_4          3  MULTIPOLYGON (((0.0020033064792571 -0.00603749...
3  6120_2_4          4                           GEOMETRYCOLLECTION EMPTY
4  6120_2_4          5  MULTIPOLYGON (((0.0091799868335186 -0.00876900...


In [25]:
df.describe()


,ClassType
count,4290.000000
mean,5.500000
std,2.872616
min,1.000000
25%,3.000000
50%,5.500000
75%,8.000000
max,10.000000
